In [30]:
import json
from textwrap import dedent

from xxhash import xxh64

from memory.cozo import client
from memory.embed import embed
from memory.generate import generate
from memory.inserts import *
from memory.queries import *
from memory.updates import *
from memory.utils import *

In [2]:
get_user_by_email("diwank@julep.ai")

{'character_id': 'd5f6a824-efbd-4945-b267-d9cd12cada77',
 'assistant_id': 'fa9dbe95-9a17-49d9-96d3-764f1970b32f',
 'assistant_about': {'name': 'Diwank',
  'about': 'Diwank is an AI researcher, entrepreneur and an amazing person.',
  'metadata': {}},
 'character_about': {'name': 'Samantha',
  'about': 'Samantha is a personal assistant, she is very diligent, incredible at her job and really really funny.',
  'metadata': {}}}

In [3]:
get_sessions_by_email("diwank@julep.ai", "")

In [29]:
chatml = [{"role": "user", "content": "Hello "*22}]
prompt = to_prompt(chatml, suffix="<|section|>me (Samantha)\n")
count_tokens(prompt)

47

In [31]:
hash = lambda d: xxh64(json.dumps(d), seed=0).hexdigest()
hash(chatml)

'75d489bd593b625f'

In [5]:
# TODO: IMPLEMENT CACHE USING COZO

In [7]:
get_human_data_time()

'01:30PM on Thursday, Jul 06 2023'

In [8]:
chatml = [
    {"role": "system", "content": f"SYSTEM"},
    *[
        {"role": "user", "content": f"Hello {i}"}
        for i in range(500)
    ]
]

truncate(chatml, max_tokens=100)

[{'role': 'system', 'content': 'SYSTEM'},
 {'role': 'user', 'content': 'Hello 497'},
 {'role': 'user', 'content': 'Hello 498'},
 {'role': 'user', 'content': 'Hello 499'}]

In [9]:
extract_keywords("Diwank's mom's name is Shikha Singh. She is an entrepreneur and a very inspiring and brave person.")

[('diwank', 0.6332),
 ('shikha', 0.5357),
 ('singh', 0.4863),
 ('mom', 0.2978),
 ('entrepreneur', 0.1408)]

In [10]:
to_belief_chatml_msg = lambda bfs: dict(role="system", name="information", content=" ".join(bfs))

chatml = [{"role": "user", "name": "Diwank", "content": "I like reading sometimes"}]
# to_belief_chatml_msg(get_matching_beliefs(chatml, 0.8))
get_matching_beliefs(chatml)

4 matching beliefs found for ```Diwank: I like reading sometimes```


['Diwank loves exploring new areas of interest.',
 "Diwank enjoys hearing other people's insights and sharing their own knowledge.",
 '- At their core, Diwank is driven by innovative ideas, products, and solutions.    - Diwank believes in being curious, open-minded, reflective, and adaptable.    - Diwank needs to visualize ambitious goals and be independent in their pursuits.',
 "Diwank doesn't care much about what others think of their shifting focus."]

In [11]:
# client.close()

### Logic for memory service

**Service contract**

```
POST /session/entry
{
  email: string,  # Required
  name: string? = "Anonymous",
  vocode_conversation_id: string, # Required
  user_input: string?,  # Only either this or situation can be passed
  situation: string?,  # If situation is passed and previous session exists raise error
}
=> {
  session_id: string,
  assistant_output: string,
}
```

**Flow**

0. `samantha = get_character(samantha_character_id)`
1. `assert situation ^ user_input`
2. `user_data = get_user_by_email(email)`
3. `if not user_data:`  
    - `character_data = create_character(name, about, assistant=samantha)`  
    - `user_data = create_user(email, character_data)`
4. `session = get_session(vocode_conversation_id)`
5. `assert situation and not session`
6. `if not session:`
    - `session = create_session(user_data, vocode_conversation_id, situation)`
    - `add_session_characters(session, user_data)`
7. `entries = get_session_entries(session_id)`
8. `{summary} = session`
9. `situation_final = make_situation(user_data, add_time=True)`
>  `make_situation = lambda data, add_time=False: (`  
>  `  f"{samantha.name} is talking to {name}."`  
>  `  + situation`  
>  `  + f'\nIt is {human_now()}'`  
>  `)`  
10.  
>  `info_top = (`  
>  `  samantha.about`  
>  `  + user_data.about`  
>  `  + f'{samantha.name} is very funny but also polite and keeps her responses brief.'`  
>  `)`  
11. `all_entries = situation_final + info_top + entries`
12. `truncated_entries = truncate(all_entries)`
13. `thought = generate_thought(truncated_entries)`
14. `beliefs = get_matching_beliefs(user_data, thought)`
15. `all_entries.append(belief)`
16. `truncated_entries = truncate(all_entries)`
17. `if len(truncated_entries) < len(all_entries) and summary:`  
    - `truncated_entries.insert(summary, 3)`
18. `assistant_output = generate(truncated_entries)`
19. `return { assistant_output, session.session_id }`
20. (In the background)  
    - `add_entry(session, assistant_output)`
    - `update_summary(session, truncated_entries[3:])`
    - `if new_session: add_episodes(session, situation)`
    - `add_episodes(session, truncated_entries)`